In [22]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import os
import pandas as pd
import numpy as np

# SSH connection details.
ssh_host = 'loki.research.cs.dal.ca'
ssh_port = 22
ssh_username = os.getenv('LOKI_USERNAME')
ssh_password = os.getenv('LOKI_PASSWORD')

# Database connection details.
db_host = '127.0.0.1'
db_port = 5432
db_name = 'staging_db'
db_user = os.getenv('STAGING_DB_USERNAME')
db_password = os.getenv('STAGING_DB_PASSWORD')

# Establish SSH tunnel and connect to PostgreSQL.
try:
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=('127.0.0.1', db_port)  # Forwarding PostgreSQL port.
    ) as tunnel:
    
        # Connect to PostgreSQL database through the SSH tunnel.
        conn = psycopg2.connect(
            host=db_host,
            port=tunnel.local_bind_port,  # use the local port set by the tunnel.
            dbname=db_name,
            user=db_user,
            password=db_password
        )
        
        print("Database connection established")
        lock_state_query = "SELECT * FROM study_prositvd.lock_state;"
        accelerometer_query = "SELECT * FROM study_prositvd.accelerometer;"
        df_lock_state = pd.read_sql_query(lock_state_query, conn)
        df_accelerometer = pd.read_sql_query(accelerometer_query, conn)
        
        # Close database connection.
        conn.close()

except Exception as e:
    print(f"An error occurred: {e}")

Database connection established


In [23]:
# Raw data.
df_lock_state.head(10)

,_id,participantid,measuredat,uploadedat,value0
0,EJRK2MdCXW29XSNKT,prositvd5,2022-11-03 17:24:54,2022-11-03 19:44:19.417,LOCKED
1,c3e3Ti9rqgLaXSHmn,prositvd5,2022-11-03 17:24:57,2022-11-03 19:44:19.417,UNLOCKED
2,GdBLBidYb44o2w6Kd,prositvd5,2022-11-03 17:26:19,2022-11-03 19:44:19.417,LOCKED
3,SaTuECPdE463Z7n26,prositvd5,2022-11-03 17:37:50,2022-11-03 19:44:19.417,UNLOCKED
4,273mh7swiQC3B9Y4F,prositvd5,2022-11-03 17:50:19,2022-11-03 19:44:19.417,LOCKED
5,iQadjzKezPNg4PWSQ,prositvd5,2022-11-03 17:50:26,2022-11-03 19:44:19.417,UNLOCKED
6,LmSjAycdKH5WgR58p,prositvd5,2022-11-03 17:54:09,2022-11-03 19:44:19.417,LOCKED
7,itv98ZhgzF34QjGtj,prositvd5,2022-11-03 17:57:14,2022-11-03 19:44:19.417,UNLOCKED
8,AbP7ccTjS9Bk8cpSi,prositvd5,2022-11-03 17:57:16,2022-11-03 19:44:19.417,LOCKED
9,uc9Q8CPcnJWEtBhde,prositvd5,2022-11-03 17:59:34,2022-11-03 19:44:19.417,UNLOCKED


In [24]:
df_accelerometer.head(10)

,_id,participantid,measuredat,uploadedat,value0,value1,value2
0,XbSWMtjaErdjG4Bxd,prositvd5,2022-11-03 17:22:59,2022-11-03 19:44:14.581,-0.196,0.242,-0.065
1,Wn3djt9CS6fRcAXg6,prositvd5,2022-11-03 17:23:00,2022-11-03 19:44:14.581,-0.007,-0.004,0.03
2,4J4f2THCPSPkjWnSN,prositvd5,2022-11-03 17:23:02,2022-11-03 19:44:14.581,0.048,0.17,-0.224
3,PBmnhHBMdAjgYuPja,prositvd5,2022-11-03 17:23:03,2022-11-03 19:44:14.581,-0.041,-0.007,0.019
4,rzfZPDAJiNkvqiMuT,prositvd5,2022-11-03 17:23:17,2022-11-03 19:44:14.581,0.13,0.166,0.228
5,HuhBwsBLPcTRG2kYc,prositvd5,2022-11-03 17:23:18,2022-11-03 19:44:14.581,0.04,-0.055,0.042
6,rjktYr7qGc5SBZxPC,prositvd5,2022-11-03 17:23:19,2022-11-03 19:44:14.581,0.175,0.038,-0.347
7,HfS6heSB2MAyMQcKw,prositvd5,2022-11-03 17:23:20,2022-11-03 19:44:14.581,0.023,0.018,-0.01
8,BKHD28tP7o43dkwGh,prositvd5,2022-11-03 17:23:50,2022-11-03 19:44:14.581,0.047,0.036,-0.313
9,xrLCtPxG9bBFCeWGL,prositvd5,2022-11-03 17:23:51,2022-11-03 19:44:14.581,0.036,0.023,0.036


## Screen time from Lock state data.

In [25]:
df_lock_state['measuredat'] = pd.to_datetime(df_lock_state['measuredat'])

# Define the logical time window.
window_start_time = pd.to_datetime('22:00:00').time()  # 10 PM
window_end_time = pd.to_datetime('07:00:00').time()    # 7 AM

# Function to assign logical day based on the window (because window can span over two calendar days).
def assign_logical_day(row):
    measured_time = row['measuredat'].time()
    
    if measured_time >= window_start_time:
        return row['measuredat'].date()  # Assign to the current day.
    else:
        return (row['measuredat'] - pd.Timedelta(days=1)).date()  # Assign to the previous day.

# Assign logical day to the DataFrame.
df_lock_state['logical_day'] = df_lock_state.apply(assign_logical_day, axis=1)

# Function to calculate screen times and metrics within logical windows.
def calculate_metrics(group):
    locked_time = pd.Timedelta(0)
    unlocked_time = pd.Timedelta(0)
    unlock_durations = []
    total_unlocks = 0

    previous_state = None
    previous_time = None
    
    # Set the logical start and end time boundaries based on the logical day.
    logical_start = pd.Timestamp.combine(group['logical_day'].iloc[0], window_start_time)
    logical_end = pd.Timestamp.combine(group['logical_day'].iloc[0] + pd.Timedelta(days=1), window_end_time)

    for _, row in group.iterrows():
        current_state = row['value0']
        current_time = row['measuredat']
        
        # Adjust previous time if crossing into the time window.
        if previous_time and previous_time < logical_start < current_time:
            previous_time = logical_start
        
        # Adjust current time if crossing the window end.
        if current_time > logical_end:
            current_time = logical_end

        # Valid transition: LOCKED -> UNLOCKED
        if previous_state == 'LOCKED' and current_state == 'UNLOCKED':
            if previous_time >= logical_start and current_time <= logical_end:
                locked_time += current_time - previous_time
        
        # Valid transition: UNLOCKED -> LOCKED
        elif previous_state == 'UNLOCKED' and current_state == 'LOCKED':
            if previous_time >= logical_start and current_time <= logical_end:
                duration = current_time - previous_time
                unlocked_time += duration
                unlock_durations.append(duration.total_seconds() / 60)  # Convert to minutes.
                total_unlocks += 1

        # Update the previous state and time.
        previous_state = current_state
        previous_time = current_time

    # Calculate statistics.
    mean_unlock = np.mean(unlock_durations) if unlock_durations else 0
    median_unlock = np.median(unlock_durations) if unlock_durations else 0
    max_unlock = max(unlock_durations) if unlock_durations else 0

    return pd.Series({
        'total_screen_locked_time': locked_time.total_seconds() / 60,
        'total_screen_unlocked_time': unlocked_time.total_seconds() / 60,
        'total_number_of_unlocks': total_unlocks,
        'mean_unlocked_duration': mean_unlock,
        'median_unlocked_duration': median_unlock,
        'max_unlocked_duration': max_unlock
    })

# Group by participant id and logical day, and apply the metric calculations.
result_df_lock_state = (
    df_lock_state.groupby(['participantid', 'logical_day'])
    .apply(calculate_metrics)
    .reset_index()
)

In [26]:
result_df_lock_state.head(10)

,participantid,logical_day,total_screen_locked_time,total_screen_unlocked_time,total_number_of_unlocks,mean_unlocked_duration,median_unlocked_duration,max_unlocked_duration
0,prositvd105,2023-02-09,0.000000,0.000000,61.0,0.000000,0.000000,0.000000
1,prositvd105,2023-02-10,280.000000,65.633333,38.0,1.727193,0.083333,58.533333
2,prositvd105,2023-02-22,296.916667,8.250000,132.0,0.062500,0.000000,1.483333
3,prositvd105,2023-02-23,432.850000,107.116667,151.0,0.709382,0.233333,13.716667
4,prositvd105,2023-02-25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,prositvd105,2023-02-27,0.000000,0.000000,185.0,0.000000,0.000000,0.000000
6,prositvd105,2023-02-28,21.750000,0.350000,2.0,0.175000,0.175000,0.183333
7,prositvd106,2023-02-15,0.000000,0.000000,35.0,0.000000,0.000000,0.000000
8,prositvd106,2023-02-16,328.900000,208.550000,221.0,0.943665,0.000000,80.466667
9,prositvd106,2023-02-17,364.416667,166.750000,40.0,4.168750,0.500000,120.166667


## Screen time from Accelerometer data

In [28]:
df_accelerometer['measuredat'] = pd.to_datetime(df_accelerometer['measuredat'])

# Convert the accelerometer values to numeric (float)
df_accelerometer['value0'] = pd.to_numeric(df_accelerometer['value0'], errors='coerce')
df_accelerometer['value1'] = pd.to_numeric(df_accelerometer['value1'], errors='coerce')
df_accelerometer['value2'] = pd.to_numeric(df_accelerometer['value2'], errors='coerce')

# Calculate movement magnitude (RMS) from accelerometer values
df_accelerometer['movement_magnitude'] = np.sqrt(df_accelerometer['value0']**2 + df_accelerometer['value1']**2 + df_accelerometer['value2']**2)

In [29]:
# Define the logical time window
window_start_time = pd.to_datetime('22:00:00').time()  # 10 PM
window_end_time = pd.to_datetime('07:00:00').time()    # 7 AM

In [30]:
# Assign logical day based on the time window.
def assign_logical_day(row):
    measured_time = row['measuredat'].time()
    
    if measured_time >= window_start_time:
        return row['measuredat'].date()  # Assign to the current day
    else:
        return (row['measuredat'] - pd.Timedelta(days=1)).date()  # Assign to the previous day

df_accelerometer['logical_day'] = df_accelerometer.apply(assign_logical_day, axis=1)

In [31]:
# Calculate the median movement magnitude for each participant and logical day.
df_accelerometer['daily_median'] = df_accelerometer.groupby(['participantid', 'logical_day'])['movement_magnitude'].transform('median')

In [32]:
# Create the 'is_asleep' column based on whether movement_magnitude is below the daily median
df_accelerometer['is_asleep'] = df_accelerometer['movement_magnitude'] < df_accelerometer['daily_median']

In [33]:
# Function to calculate sleep times and other metrics, similar to how we handled the lock_state data.
def calculate_sleep_metrics(group):
    sleep_time = pd.Timedelta(0)
    wake_time = pd.Timedelta(0)
    sleep_durations = []
    total_sleep_segments = 0

    previous_state = None
    previous_time = None
    
    # Set the logical start and end time boundaries based on the logical day
    logical_start = pd.Timestamp.combine(group['logical_day'].iloc[0], window_start_time)
    logical_end = pd.Timestamp.combine(group['logical_day'].iloc[0] + pd.Timedelta(days=1), window_end_time)

    for _, row in group.iterrows():
        current_state = row['is_asleep']
        current_time = row['measuredat']
        
        # Adjust previous time if crossing into the time window.
        if previous_time and previous_time < logical_start < current_time:
            previous_time = logical_start
        
        # Adjust current time if crossing the window end.
        if current_time > logical_end:
            current_time = logical_end

        # Transition: Awake -> Asleep
        if previous_state == False and current_state == True:
            if previous_time >= logical_start and current_time <= logical_end:
                wake_time += current_time - previous_time
        
        # Transition: Asleep -> Awake
        elif previous_state == True and current_state == False:
            if previous_time >= logical_start and current_time <= logical_end:
                duration = current_time - previous_time
                sleep_time += duration
                sleep_durations.append(duration.total_seconds() / 60)  # Convert to minutes.
                total_sleep_segments += 1

        # Update the previous state and time.
        previous_state = current_state
        previous_time = current_time

    # Calculate statistics.
    mean_sleep_duration = np.mean(sleep_durations) if sleep_durations else 0
    median_sleep_duration = np.median(sleep_durations) if sleep_durations else 0
    max_sleep_duration = max(sleep_durations) if sleep_durations else 0

    return pd.Series({
        'total_sleep_time': sleep_time.total_seconds() / 60,
        'total_wake_time': wake_time.total_seconds() / 60,
        'total_sleep_segments': total_sleep_segments,
        'mean_sleep_duration': mean_sleep_duration,
        'median_sleep_duration': median_sleep_duration,
        'max_sleep_duration': max_sleep_duration
    })

# Group by participant id and logical day, and apply the metric calculations.
result_df_accel = (
    df_accelerometer.groupby(['participantid', 'logical_day'])
    .apply(calculate_sleep_metrics)
    .reset_index()
)

In [34]:
result_df_accel.head(10)

,participantid,logical_day,total_sleep_time,total_wake_time,total_sleep_segments,mean_sleep_duration,median_sleep_duration,max_sleep_duration
0,prositvd105,2023-02-09,0.000000,0.000000,867.0,0.000000,0.000000,0.000000
1,prositvd105,2023-02-10,146.100000,148.816667,642.0,0.227570,0.016667,53.233333
2,prositvd105,2023-02-22,269.866667,3.583333,2741.0,0.098456,0.000000,55.933333
3,prositvd105,2023-02-23,445.616667,27.950000,1472.0,0.302729,0.050000,22.850000
4,prositvd105,2023-02-25,0.083333,16.000000,8.0,0.010417,0.000000,0.050000
5,prositvd105,2023-02-27,0.000000,0.000000,1781.0,0.000000,0.000000,0.000000
6,prositvd105,2023-02-28,23.233333,2.000000,76.0,0.305702,0.108333,2.850000
7,prositvd106,2023-02-15,0.000000,0.000000,732.0,0.000000,0.000000,0.000000
8,prositvd106,2023-02-16,409.616667,16.616667,3157.0,0.129749,0.000000,63.666667
9,prositvd106,2023-02-17,497.800000,9.016667,564.0,0.882624,0.033333,134.183333


In [35]:
result_df_lock_state.head(10)

,participantid,logical_day,total_screen_locked_time,total_screen_unlocked_time,total_number_of_unlocks,mean_unlocked_duration,median_unlocked_duration,max_unlocked_duration
0,prositvd105,2023-02-09,0.000000,0.000000,61.0,0.000000,0.000000,0.000000
1,prositvd105,2023-02-10,280.000000,65.633333,38.0,1.727193,0.083333,58.533333
2,prositvd105,2023-02-22,296.916667,8.250000,132.0,0.062500,0.000000,1.483333
3,prositvd105,2023-02-23,432.850000,107.116667,151.0,0.709382,0.233333,13.716667
4,prositvd105,2023-02-25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,prositvd105,2023-02-27,0.000000,0.000000,185.0,0.000000,0.000000,0.000000
6,prositvd105,2023-02-28,21.750000,0.350000,2.0,0.175000,0.175000,0.183333
7,prositvd106,2023-02-15,0.000000,0.000000,35.0,0.000000,0.000000,0.000000
8,prositvd106,2023-02-16,328.900000,208.550000,221.0,0.943665,0.000000,80.466667
9,prositvd106,2023-02-17,364.416667,166.750000,40.0,4.168750,0.500000,120.166667
